# CI7340 - ADP — Data Cleaning & Transformation - Kevin Gomez
**Dataset:** Montgomery County Crime (2018–2022)  
**Goal:** Inspect → Clean → Deduplicate → Add helper fields → Save clean dataset + brief reports

In [23]:
from __future__ import annotations
import io
from pathlib import Path
import numpy as np
import pandas as pd, pyarrow
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings("ignore", message="Could not infer format")

RAW_CSV_PATH = "Crime_Dataset_Coursework1-1.csv"

OUT_DIR_DATA = Path("data/processed")
OUT_DIR_REPORTS = Path("reports")
OUT_DIR_FIGS = Path("reports/figs")

for d in [OUT_DIR_DATA, OUT_DIR_REPORTS, OUT_DIR_FIGS]:
    d.mkdir(parents=True, exist_ok=True)

CLEAN_CSV_PATH = OUT_DIR_DATA / "crime_clean.csv"
CLEAN_PARQUET_PATH = OUT_DIR_DATA / "crime_clean.parquet"
MISSING_BEFORE_CSV = OUT_DIR_REPORTS / "missingness_before.csv"
MISSING_AFTER_CSV = OUT_DIR_REPORTS / "missingness_after.csv"
DQ_SUMMARY_MD = OUT_DIR_REPORTS / "data_quality_summary.md"
DATA_DICTIONARY_MD = OUT_DIR_REPORTS / "data_dictionary.md"

DATE_COLUMNS = ["Dispatch Date / Time", "Start_Date_Time", "End_Date_Time"]
OPTIONALLY_DROP_IF_VERY_SPARSE = ["Street Prefix", "Street Suffix"]
YEAR_MIN, YEAR_MAX = 2018, 2022

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 200)

In [3]:
df_raw = pd.read_csv(RAW_CSV_PATH, low_memory=False)
print(df_raw.shape)
df_raw.head()

(306094, 30)


,Incident ID,Offence Code,CR Number,Dispatch Date / Time,NIBRS Code,Victims,Crime Name1,Crime Name2,Crime Name3,Police District Name,Block Address,City,State,Zip Code,Agency,Place,Sector,Beat,PRA,Address Number,Street Prefix,Street Name,Street Suffix,Street Type,Start_Date_Time,End_Date_Time,Latitude,Longitude,Police District Number,Location
0,201202980,3550,180042096,08/23/2018 09:52:08 PM,35B,1,Crime Against Society,Drug Equipment Violations,DRUGS - NARCOTIC EQUIP - POSSESS,GERMANTOWN,12800 BLK MIDDLEBROOK RD,GERMANTOWN,MD,20874.0,MCPD,Street - In vehicle,N,5N1,447,12800.0,NaN,MIDDLEBROOK,NaN,RD,08/23/2018 09:52:00 PM,NaN,39.177744,-77.265619,5D,"(39.1777, -77.2656)"
1,201181293,3522,180015424,03/30/2018 01:00:55 AM,35A,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - OPIUM OR DERIVATIVE - POSSESS,BETHESDA,8300 BLK WOODMONT AVE,BETHESDA,MD,20814.0,MCPD,Street - In vehicle,E,2E2,054,8300.0,NaN,WOODMONT,NaN,AVE,03/30/2018 01:01:00 AM,NaN,38.992693,-77.097063,2D,"(38.9927, -77.0971)"
2,201181293,3562,180015424,03/30/2018 01:00:55 AM,35A,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - MARIJUANA - POSSESS,BETHESDA,8300 BLK WOODMONT AVE,BETHESDA,MD,20814.0,MCPD,Street - In vehicle,E,2E2,054,8300.0,NaN,WOODMONT,NaN,AVE,03/30/2018 01:01:00 AM,NaN,38.992693,-77.097063,2D,"(38.9927, -77.0971)"
3,201193163,3520,180029476,06/14/2018 10:26:45 PM,35A,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - OPIUM OR DERIVATIVE - SELL,MONTGOMERY VILLAGE,400 BLK QUINCE ORCHARD RD,GAITHERSBURG,MD,20878.0,GPD,Hotel/Motel/Etc.,P,6P2,445,400.0,NaN,QUINCE ORCHARD,NaN,RD,06/14/2018 10:26:00 PM,06/15/2018 03:00:00 AM,39.147954,-77.218189,6D,"(39.148, -77.2182)"
4,201204355,2204,180043926,09/03/2018 12:06:54 PM,220,1,Crime Against Property,Burglary/Breaking and Entering,BURGLARY - NO FORCED ENTRY-RESIDENTIAL,BETHESDA,4800 BLK FALSTONE AVE,CHEVY CHASE,MD,20815.0,MCPD,Residence - Single Family,E,2E1,017,4800.0,NaN,FALSTONE,NaN,AVE,09/02/2018 11:30:00 PM,09/03/2018 12:30:00 AM,38.966174,-77.096561,2D,"(38.9662, -77.0966)"


In [4]:
df_raw.info()
df_raw.columns.to_list()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306094 entries, 0 to 306093
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Incident ID             306094 non-null  int64  
 1   Offence Code            306094 non-null  object 
 2   CR Number               306094 non-null  int64  
 3   Dispatch Date / Time    257065 non-null  object 
 4   NIBRS Code              306094 non-null  object 
 5   Victims                 306094 non-null  int64  
 6   Crime Name1             305822 non-null  object 
 7   Crime Name2             305822 non-null  object 
 8   Crime Name3             305822 non-null  object 
 9   Police District Name    306000 non-null  object 
 10  Block Address           279888 non-null  object 
 11  City                    304818 non-null  object 
 12  State                   306094 non-null  object 
 13  Zip Code                302915 non-null  float64
 14  Agency              

['Incident ID',
 'Offence Code',
 'CR Number',
 'Dispatch Date / Time',
 'NIBRS Code',
 'Victims',
 'Crime Name1',
 'Crime Name2',
 'Crime Name3',
 'Police District Name',
 'Block Address',
 'City',
 'State',
 'Zip Code',
 'Agency',
 'Place',
 'Sector',
 'Beat',
 'PRA',
 'Address Number',
 'Street Prefix',
 'Street Name',
 'Street Suffix',
 'Street Type',
 'Start_Date_Time',
 'End_Date_Time',
 'Latitude',
 'Longitude',
 'Police District Number',
 'Location']

In [5]:
miss_before = df_raw.isna().mean().sort_values(ascending=False).to_frame("missing_ratio")
miss_before["missing_pct"] = (miss_before["missing_ratio"] * 100).round(2)
miss_before.to_csv(MISSING_BEFORE_CSV)
miss_before.head(15)

,missing_ratio,missing_pct
Street Suffix,0.982254,98.23
Street Prefix,0.955468,95.55
End_Date_Time,0.528132,52.81
Dispatch Date / Time,0.160176,16.02
Block Address,0.085614,8.56
Address Number,0.085297,8.53
Zip Code,0.010386,1.04
Sector,0.004998,0.50
Beat,0.004998,0.50
City,0.004169,0.42


In [6]:
def normalise_text_series(s: pd.Series) -> pd.Series:
    s2 = s.astype("string")
    s2 = s2.str.strip().str.replace(r"\s+", " ", regex=True)
    s2 = s2.where(s2.isna(), s2.str.title())
    return s2

def to_datetime_safe(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce")

def coerce_zip(zip_series: pd.Series) -> pd.Series:
    z = zip_series.copy()
    if pd.api.types.is_float_dtype(z):
        z = z.round().astype("Int64")
    z = z.astype("string")
    z = z.str.replace(r"[^\d]", "", regex=True)
    z = z.where(z.isna() | (z.str.len() == 5), z.str.zfill(5))
    z = z.replace("", pd.NA)
    return z

def choose_primary_offense(row: pd.Series):
    for col in ["Crime Name1", "Crime Name2", "Crime Name3"]:
        if col in row and pd.notna(row[col]) and str(row[col]).strip():
            return str(row[col]).strip()
    return pd.NA

In [7]:
df = df_raw.copy()

# Parse any date columns that exist
for dc in [c for c in DATE_COLUMNS if c in df.columns]:
    df[dc] = to_datetime_safe(df[dc])

df[["Dispatch Date / Time", "Start_Date_Time", "End_Date_Time"]].head(3)

,Dispatch Date / Time,Start_Date_Time,End_Date_Time
0,2018-08-23 21:52:08,2018-08-23 21:52:00,NaT
1,2018-03-30 01:00:55,2018-03-30 01:01:00,NaT
2,2018-03-30 01:00:55,2018-03-30 01:01:00,NaT


In [8]:
# Normalise common text columns if present
for c in ["Offence Code","NIBRS Code","Crime Name1","Crime Name2","Crime Name3",
          "Police District Name","City","State","Agency","Place","Sector","Beat","PRA",
          "Street Prefix","Street Name","Street Suffix","Street Type"]:
    if c in df.columns:
        df[c] = normalise_text_series(df[c])

# Zip Code cleanup
if "Zip Code" in df.columns:
    df["Zip Code"] = coerce_zip(df["Zip Code"])

# Non-negative victims
if "Victims" in df.columns:
    df.loc[df["Victims"] < 0, "Victims"] = pd.NA

# Latitude/Longitude sanity
if "Latitude" in df.columns:
    df.loc[~df["Latitude"].between(-90, 90), "Latitude"] = pd.NA
if "Longitude" in df.columns:
    df.loc[~df["Longitude"].between(-180, 180), "Longitude"] = pd.NA

# Primary offense from Crime Name1/2/3
df["Primary Offense"] = df.apply(choose_primary_offense, axis=1).astype("string")
df["Primary Offense"].head(3)

0    Crime Against Society
1    Crime Against Society
2    Crime Against Society
Name: Primary Offense, dtype: string

In [9]:
# Drop super-sparse columns (>90% missing) if listed
miss_ratio = df.isna().mean()
to_drop = [c for c in OPTIONALLY_DROP_IF_VERY_SPARSE if c in df.columns and miss_ratio[c] > 0.90]
if to_drop:
    df = df.drop(columns=to_drop)
to_drop

# Categorical-like → fill "Unknown"; numeric → median; dates → leave as NaT
for c in df.columns:
    if c in DATE_COLUMNS:
        continue
    if pd.api.types.is_object_dtype(df[c]) or pd.api.types.is_string_dtype(df[c]):
        df[c] = df[c].fillna("Unknown")
    elif pd.api.types.is_numeric_dtype(df[c]):
        if df[c].isna().any():
            med = df[c].median() if df[c].notna().any() else None
            if med is not None:
                df[c] = df[c].fillna(med)

df.head(3)

,Incident ID,Offence Code,CR Number,Dispatch Date / Time,NIBRS Code,Victims,Crime Name1,Crime Name2,Crime Name3,Police District Name,Block Address,City,State,Zip Code,Agency,Place,Sector,Beat,PRA,Address Number,Street Name,Street Type,Start_Date_Time,End_Date_Time,Latitude,Longitude,Police District Number,Location,Primary Offense
0,201202980,3550,180042096,2018-08-23 21:52:08,35B,1.0,Crime Against Society,Drug Equipment Violations,Drugs - Narcotic Equip - Possess,Germantown,12800 BLK MIDDLEBROOK RD,Germantown,Md,20874,Mcpd,Street - In Vehicle,N,5N1,447,12800.0,Middlebrook,Rd,2018-08-23 21:52:00,NaT,39.177744,-77.265619,5D,"(39.1777, -77.2656)",Crime Against Society
1,201181293,3522,180015424,2018-03-30 01:00:55,35A,1.0,Crime Against Society,Drug/Narcotic Violations,Drugs - Opium Or Derivative - Possess,Bethesda,8300 BLK WOODMONT AVE,Bethesda,Md,20814,Mcpd,Street - In Vehicle,E,2E2,054,8300.0,Woodmont,Ave,2018-03-30 01:01:00,NaT,38.992693,-77.097063,2D,"(38.9927, -77.0971)",Crime Against Society
2,201181293,3562,180015424,2018-03-30 01:00:55,35A,1.0,Crime Against Society,Drug/Narcotic Violations,Drugs - Marijuana - Possess,Bethesda,8300 BLK WOODMONT AVE,Bethesda,Md,20814,Mcpd,Street - In Vehicle,E,2E2,054,8300.0,Woodmont,Ave,2018-03-30 01:01:00,NaT,38.992693,-77.097063,2D,"(38.9927, -77.0971)",Crime Against Society


In [10]:
# Choose a date column for year filtering
date_for_year = "Start_Date_Time" if "Start_Date_Time" in df.columns else ("Dispatch Date / Time" if "Dispatch Date / Time" in df.columns else None)

rows_before = len(df)
if date_for_year:
    df[date_for_year] = to_datetime_safe(df[date_for_year])
    yrs = df[date_for_year].dt.year
    df = df[(yrs >= YEAR_MIN) & (yrs <= YEAR_MAX)]
rows_after = len(df)
rows_before, rows_after

(306094, 221565)

In [11]:
if "Incident ID" in df.columns:
    # Prefer to keep the "latest" using End then Dispatch if available
    sort_cols = [c for c in ["End_Date_Time","Dispatch Date / Time"] if c in df.columns]
    if sort_cols:
        df = df.sort_values(sort_cols)
    before = len(df)
    df = df.drop_duplicates(subset=["Incident ID"], keep="last")
    after = len(df)
    print(f"Deduplicated by Incident ID: {before} -> {after}")
else:
    # Fall back to full row duplicate removal
    before = len(df)
    df = df.drop_duplicates()
    after = len(df)
    print(f"Deduplicated exact duplicates: {before} -> {after}")

Deduplicated by Incident ID: 221565 -> 203304


In [12]:
if date_for_year:
    df["year"] = df[date_for_year].dt.year
    df["month"] = df[date_for_year].dt.month
    df["weekday"] = df[date_for_year].dt.day_name()
    df["hour"] = df[date_for_year].dt.hour

if "Police District Name" in df.columns:
    df["District_Norm"] = normalise_text_series(df["Police District Name"])

df[["year","month","weekday","hour"]].head(3)

,year,month,weekday,hour
186095,2018,1,Monday,0
216018,2018,1,Monday,12
172934,2018,1,Monday,2


In [13]:
miss_after = df.isna().mean().sort_values(ascending=False).to_frame("missing_ratio")
miss_after["missing_pct"] = (miss_after["missing_ratio"] * 100).round(2)

miss_after.to_csv(MISSING_AFTER_CSV)
print("Saved:", MISSING_AFTER_CSV)

display(miss_before.head(12))
display(miss_after.head(12))

Saved: reports/missingness_after.csv


,missing_ratio,missing_pct
Street Suffix,0.982254,98.23
Street Prefix,0.955468,95.55
End_Date_Time,0.528132,52.81
Dispatch Date / Time,0.160176,16.02
Block Address,0.085614,8.56
Address Number,0.085297,8.53
Zip Code,0.010386,1.04
Sector,0.004998,0.50
Beat,0.004998,0.50
City,0.004169,0.42


,missing_ratio,missing_pct
End_Date_Time,0.543777,54.38
Dispatch Date / Time,0.031175,3.12
Incident ID,0.000000,0.00
Police District Number,0.000000,0.00
Street Name,0.000000,0.00
Street Type,0.000000,0.00
Start_Date_Time,0.000000,0.00
Latitude,0.000000,0.00
Longitude,0.000000,0.00
Location,0.000000,0.00


In [14]:
df.to_csv(CLEAN_CSV_PATH, index=False)
print("Clean CSV:", CLEAN_CSV_PATH)

# Parquet (optional, fast) — requires pyarrow or fastparquet
try:
    df.to_parquet(CLEAN_PARQUET_PATH, index=False)
    print("Clean Parquet:", CLEAN_PARQUET_PATH)
except Exception as e:
    print("Parquet not written (install pyarrow/fastparquet). Error:", e)

Clean CSV: data/processed/crime_clean.csv
Clean Parquet: data/processed/crime_clean.parquet


In [15]:
paths = []

if "year" in df.columns:
    by_year = df.groupby("year").size().rename("count").reset_index()
    p = OUT_DIR_DATA / "agg_by_year.csv"; by_year.to_csv(p, index=False); paths.append(p)

if "District_Norm" in df.columns:
    by_dist = df.groupby("District_Norm").size().rename("count").reset_index()
    p = OUT_DIR_DATA / "agg_by_district.csv"; by_dist.to_csv(p, index=False); paths.append(p)

if "Primary Offense" in df.columns:
    by_off = df.groupby("Primary Offense").size().rename("count").reset_index()
    p = OUT_DIR_DATA / "agg_by_offense.csv"; by_off.to_csv(p, index=False); paths.append(p)

paths

[PosixPath('data/processed/agg_by_year.csv'),
 PosixPath('data/processed/agg_by_district.csv'),
 PosixPath('data/processed/agg_by_offense.csv')]

In [20]:
def build_data_dictionary(df: pd.DataFrame) -> str:
    lines = ["# Data Dictionary\n",
             f"- Rows: **{len(df):,}**  |  Columns: **{df.shape[1]}**\n",
             "| Column | Dtype | Example |",
             "|---|---|---|"]
    for c in df.columns:
        dt = str(df[c].dtype)
        ex = df[c].dropna().iloc[0] if df[c].notna().any() else "—"
        lines.append(f"| {c} | {dt} | {ex} |")
    return "\n".join(lines)

def build_dq_summary(before: pd.DataFrame, after: pd.DataFrame) -> str:
    rows_before, rows_after = len(before), len(after)
    miss_before = int(before.isna().sum().sum())
    miss_after = int(after.isna().sum().sum())
    reduction = miss_before - miss_after

    notes = [
        "- Categorical NAs filled as 'Unknown' (transparent & explainable).",
        "- Numeric NAs filled with median; dates left as NaT when unknown.",
        f"- Year window enforced: {YEAR_MIN}–{YEAR_MAX} based on available date column.",
        "- Deduplicated by Incident ID (kept latest where timestamps available).",
        "- Very sparse columns (e.g., Street Prefix/Suffix) dropped if >90% missing."
    ]
    return "\n".join([
        "# Data Quality Summary",
        "",
        f"- Rows: **{rows_before:,} → {rows_after:,}**",
        f"- Missing cells (total): **{miss_before:,} → {miss_after:,}** (**{reduction:,}** fewer)",
        "",
        "## Notes",
        *notes
    ])

DATA_DICTIONARY_MD.write_text(build_data_dictionary(df), encoding="utf-8")
DQ_SUMMARY_MD.write_text(build_dq_summary(df_raw, df), encoding="utf-8")

print("Wrote:")
print("-", DATA_DICTIONARY_MD)
print("-", DQ_SUMMARY_MD)

Wrote:
- reports/data_dictionary.md
- reports/data_quality_summary.md


In [21]:
display(Markdown(open("reports/data_dictionary.md").read()))

# Data Dictionary

- Rows: **203,304**  |  Columns: **34**

| Column | Dtype | Example |
|---|---|---|
| Incident ID | int64 | 201175193 |
| Offence Code | string | 2399 |
| CR Number | int64 | 180007927 |
| Dispatch Date / Time | datetime64[ns] | 2018-02-15 12:54:11 |
| NIBRS Code | string | 23H |
| Victims | float64 | 1.0 |
| Crime Name1 | string | Crime Against Property |
| Crime Name2 | string | All Other Larceny |
| Crime Name3 | string | Larceny (Describe Offense) |
| Police District Name | string | Silver Spring |
| Block Address | object | 11200 BLK  NEW HAMPSHIRE AVE |
| City | string | Silver Spring |
| State | string | Md |
| Zip Code | string | 20904 |
| Agency | string | Mcpd |
| Place | string | Street - Commercial |
| Sector | string | I |
| Beat | string | 3I1 |
| PRA | string | 518 |
| Address Number | float64 | 11200.0 |
| Street Name | string | New Hampshire |
| Street Type | string | Ave |
| Start_Date_Time | datetime64[ns] | 2018-01-01 00:00:00 |
| End_Date_Time | datetime64[ns] | 2018-01-01 00:00:00 |
| Latitude | float64 | 39.039818289 |
| Longitude | float64 | -76.990615251 |
| Police District Number | object | 3D |
| Location | object | (39.0398, -76.9906) |
| Primary Offense | string | Crime Against Property |
| year | int32 | 2018 |
| month | int32 | 1 |
| weekday | object | Monday |
| hour | int32 | 0 |
| District_Norm | string | Silver Spring |

In [22]:
display(Markdown(open("reports/data_quality_summary.md").read()))

# Data Quality Summary

- Rows: **306,094 → 203,304**
- Missing cells (total): **865,131 → 116,890** (**748,241** fewer)

## Notes
- Categorical NAs filled as 'Unknown' (transparent & explainable).
- Numeric NAs filled with median; dates left as NaT when unknown.
- Year window enforced: 2018–2022 based on available date column.
- Deduplicated by Incident ID (kept latest where timestamps available).
- Very sparse columns (e.g., Street Prefix/Suffix) dropped if >90% missing.

## Hand-off items for the team
- Clean dataset: `data/processed/crime_clean.csv` (+ optional Parquet)
- Quick aggregates: year, district, offense
- Reports:
  - `reports/data_dictionary.md`
  - `reports/missingness_before.csv`, `reports/missingness_after.csv`
  - `reports/data_quality_summary.md`

**Next (team):** Use clean dataset for 10 research questions × 2 visuals each (20 total).